In [41]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.cross_validation import train_test_split
from ml_metrics import rmsle
import xgboost as xgb
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model, preprocessing
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix
from scipy import io
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [42]:
products = pd.read_csv("../input/producto_tabla.csv")

In [43]:
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
products['weight'] = w[0].astype('float') * w[1].map({'Kg': 1000, 'g': 1})
products['pieces'] = products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')

products['short_name_processed'] = (products['short_name'].
                                    map(lambda x: " ".
                                        join([i for i in x.lower().split() 
                                              if i not in nltk.corpus.stopwords.words("spanish")])))
stemmer = SnowballStemmer("spanish")
products['short_name_processed'] = (products['short_name_processed'].
                                    map(lambda x: " ".join([stemmer.stem(i) for i in x.lower().split()])))

le = preprocessing.LabelEncoder()

products['Prod_name_ID']=le.fit_transform(products['short_name_processed'])
products['Brand_ID']=le.fit_transform(products['brand'])

products = products.drop(['short_name', 'brand', 'short_name_processed', 'NombreProducto'], axis=1)
products.fillna(value=0, inplace=True)
print('products shape:', products.shape)

('products shape:', (2592, 5))


In [44]:
products.describe()

,Producto_ID,weight,pieces,Prod_name_ID,Brand_ID
count,2592.000000,2592.000000,2592.000000,2592.000000,2592.000000
mean,32591.095679,396.512731,4.554012,493.928627,19.851852
std,13004.091024,1746.272566,8.988187,285.759610,12.819128
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,31515.500000,70.000000,0.000000,249.000000,4.000000
50%,35224.000000,170.000000,0.000000,498.000000,23.000000
75%,40447.250000,384.000000,6.000000,750.250000,25.000000
max,49997.000000,42000.000000,200.000000,962.000000,44.000000


In [45]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 5 columns):
Producto_ID     2592 non-null int64
weight          2592 non-null float64
pieces          2592 non-null float64
Prod_name_ID    2592 non-null int64
Brand_ID        2592 non-null int64
dtypes: float64(2), int64(3)
memory usage: 101.3 KB


In [46]:
products[['pieces','Brand_ID']] = products[['pieces','Brand_ID']].astype('uint8')
products[['Producto_ID','weight', 'Prod_name_ID']] = products[['Producto_ID','weight', 'Prod_name_ID']].astype('uint16')

In [47]:
products.describe()

,Producto_ID,weight,pieces,Prod_name_ID,Brand_ID
count,2592.000000,2592.000000,2592.000000,2592.000000,2592.000000
mean,32591.095679,396.512731,4.554012,493.928627,19.851852
std,13004.091024,1746.272566,8.988187,285.759610,12.819128
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,31515.500000,70.000000,0.000000,249.000000,4.000000
50%,35224.000000,170.000000,0.000000,498.000000,23.000000
75%,40447.250000,384.000000,6.000000,750.250000,25.000000
max,49997.000000,42000.000000,200.000000,962.000000,44.000000


In [48]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 5 columns):
Producto_ID     2592 non-null uint16
weight          2592 non-null uint16
pieces          2592 non-null uint8
Prod_name_ID    2592 non-null uint16
Brand_ID        2592 non-null uint8
dtypes: uint16(3), uint8(2)
memory usage: 20.3 KB


In [49]:
products.to_csv('../input/products_ID.csv')